### Notebook to be used to test that ATCamera, ATHeaderService and ATArchiver are online and working

In [1]:
import numpy as np
from lsst.ts import salobj
import asyncio
from astropy.io import fits

import warnings
#import matplotlib.pyplot as plt  # imported as py above
from astropy.modeling import models, fitting
from scipy.ndimage.filters import gaussian_filter
from matplotlib import pyplot as plt
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')
import time
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.geom

import os
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
logger = logging.getLogger('image_display_notebook')




In [2]:
# Want to display in firefly?
# first open a tunnel
## ssh -L 8000:10.0.100.230:8080 pingraham@140.252.32.143
afwDisplay.setDefaultBackend('firefly')
os.environ['FIREFLY_HTML'] = "slate.html"
os.environ['FIREFLY_URL'] = 'http://10.0.100.230:8080/firefly'
# now open a tab and navigate to http://localhost:8000/firefly/

In [3]:
# # Setup the butler
#accs_images = True
accs_images = False
if accs_images:
    repo = os.path.join("/home/saluser/ingest/accs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    butler = dafPersist.Butler(repo)
else:
    #repo = os.path.join("/home/saluser/ingest/dmcs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    repo = os.path.join("/home/saluser/ingest/dmcs/")
    butler = dafPersist.Butler(repo) #
    #butler = dafPersist.Butler(repo, mapper="lsst.obs.lsst.auxTel.AuxTelMapper")
#test   
raw = butler.get("raw", visit=2019111300004)
image = raw.getImage().array

In [4]:
d = salobj.Domain()

ATArchiver = salobj.Remote(d, 'ATArchiver')
await ATArchiver.start_task
ATCamera = salobj.Remote(d, 'ATCamera')
await ATCamera.start_task
#ATMonochromator = salobj.Remote(SALPY_ATMonochromator)


2019-11-13 17:45:42,130 ATArchiver   INFO     Read historical data in 11.17 sec
2019-11-13 17:45:44,409 ATCamera     INFO     Read historical data in 0.01 sec


In [5]:
from lsst.ts.standardscripts.auxtel.latiss import LATISS

latiss = LATISS(salobj.Remote(domain=d, name="ATCamera"), 
                salobj.Remote(domain=d, name="ATSpectrograph"))

await asyncio.gather(latiss.atcam.start_task, latiss.atspec.start_task)

2019-11-13 17:45:48,376 ATCamera     INFO     Read historical data in 0.01 sec
2019-11-13 17:45:48,781 ATSpectrograph INFO     Read historical data in 0.41 sec


[None, None]

In [6]:
#tmp = await latiss.take_bias(1)

### Leave this cell for state transitions 

In [7]:
# tmp=await ATArchiver.cmd_enable.start(timeout=10)
# tmp.result

In [8]:
#  bring into enable state if required
if True:
    try:
        await ATCamera.cmd_enable.start(timeout=15)
    except salobj.AckError as ack_err:
        tmp=ack_err
        print(f"Failed withs ack={tmp.ackcmd.ack},ack.error={tmp.ackcmd.error}, ackcmd.result={tmp.ackcmd.result}")
else:
    print('Ignoring')
#asyncio.get_event_loop().run_until_complete(ATCamera.cmd_start.start())

Failed withs ack=-300,ack.error=0, ackcmd.result=Ack : NO Command not accepted in State{SummaryState = ENABLED}


In [154]:
# Take image

#First flush events that we want to listen to
ATArchiver.evt_processingStatus.flush()
ATCamera.evt_endReadout.flush()

group_id='Test'
expTime=60.0
ATCamera.cmd_takeImages.set(expTime=expTime, shutter=0, numImages=1, imageType='DARK', groupId='test')

try:
    await ATCamera.cmd_takeImages.start(timeout=expTime+15)
except salobj.AckError as ack_err:
    print(f"Failed withs ack={ack_err.ackcmd.ack},ack.error={ack_err.ackcmd.error}, ackcmd.result={ack_err.ackcmd.result}")

endReadout = await ATCamera.evt_endReadout.next(flush=False, timeout=30)
print('ATCamera Wrote file {}'.format(endReadout.imageName) )
# wait for Archiver
test= await ATArchiver.evt_processingStatus.next(flush=False, timeout=30)
print(test.description)

2019-11-13 19:07:05,983 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 75 messages
2019-11-13 19:07:06,042 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 75 messages
2019-11-13 19:07:06,098 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, logMessage) falling behind; read 100 messages


ATCamera Wrote file AT_O_20191113_000027
AT_O_20191113_000026--R00S00.fits is successfuly transferred to ARC@10.0.100.128:/data/staging/atforwarder/2019-11-13/AT_O_20191113_000026--R00S00.fits


In [155]:
# # Read the file manually
# dir = '/mnt/data/ats/mcm/20191113/'
# atcamera_fname=endReadout.imageName

# full_file_name=dir+atcamera_fname+'.fits'
# print('Opening {}'.format(full_file_name))
# hdu_list = fits.open(full_file_name)

In [156]:
#hdu_list[0].header

In [157]:
# data = hdu_list[7].data
# plt.hist(data[100:300,100:300], bins=10) 
# #plot.axis([50, 110, 0, 0.06]) 
# #axis([xmin,xmax,ymin,ymax])
# plt.xlabel('pixel value')
# plt.ylabel('Occurances')
# plt.imshow(data)

In [158]:
import importlib
import utils.processExposure 
importlib.reload(utils.processExposure)
from utils.processExposure import processExposure

# import utils.grabATImage 
# importlib.reload(utils.grabATImage)
# from utils.grabATImage import grabATImage

In [159]:
# parse out visitID from filename - this is highly annoying
tmp=endReadout.imageName.split('_')
prefix=tmp[2] # dayobs without the dashes

# Don't remember why I used int here... whitespace? 
# surely fixable but bigger fish.
suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
visitID = int((prefix+suffix))
print(visitID)

# Grab image from butler, but need to wait to ingestion so use this polling function
#exposure = await grabATImage(visitID, repo, timeout = 40, poll_freq_hz=2)

#visitID = int(2019111300021)
# have to redefine butler after each image
butler = dafPersist.Butler(repo)
exposure = butler.get("raw", visit=visitID)
#image = raw.getImage().array

# do ISR correction
isr_corr_exposure = processExposure(exposure, repo=repo)
#isr_corr_exposure = exposure

2019111300027


NoResults: No locations for get: datasetType:raw_hdu dataId:DataId(initialdata={'visit': 2019111300027}, tag=set())

In [ ]:
# Only here for me to debug, should be up top with other declarations
import importlib
import utils.findNarrowbandRonchiPeaks
importlib.reload(utils.findNarrowbandRonchiPeaks)
from utils.findNarrowbandRonchiPeaks import findNarrowbandRonchiPeaks

import utils.fitExposure
importlib.reload(utils.fitExposure)
from utils.fitExposure import fit2DGaussian

import utils.calc_CofM
importlib.reload(utils.calc_CofM)
from utils.calc_CofM import calc_CofM

import utils.calc_encircled_energy
importlib.reload(utils.calc_encircled_energy)
from utils.calc_encircled_energy import calc_encircled_energy

In [160]:
# Source detection libraries
from lsst.meas.algorithms.detection import SourceDetectionTask
import lsst.afw.table as afwTable

# create the output table for source detection
schema = afwTable.SourceTable.makeMinimalSchema()
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10  # detection threshold after smoothing
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

In [161]:
if True: # display the image in firefly
    plt.close('all')
    disp = afwDisplay.Display(2, reopenPlot=True)
    disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
    disp.setImageColormap('gray')
    disp.scale('linear', 'zscale')
    disp.mtv(isr_corr_exposure, title='visit = {}'.format(visitID))
    #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

2019-11-13 19:08:35,361 requests.packages.urllib3.connectionpool INFO     Starting new HTTP connection (1): 10.0.100.230


In [162]:
# I don't like looping but I don't know how to handle multiple files yet!
# Declare approximation of where the zero-order star is
zeroth_order_estimate = lsst.geom.Point2D(1700,1960)
zeroth_order_estimate = lsst.geom.Point2D(1650,1930)
zeroth_order_estimate = lsst.geom.Point2D(2100,2100)

fit_data=[]

for index, img_name in enumerate(image_list):
    # parse out visitID from filename - this is highly annoying
    print('Processing file {} of {}, filename={}'.format(index,len(image_list), img_name))
    tmp=img_name.split('_')
    prefix=tmp[2] # dayobs without the dashes
    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    dataId1 = {'visit': visitID}
    #multi_file_dataset[i]['visitID']=visitID
    
    #exposure = butler.get('raw', **dataId1)
    exposure = await grabATImage(visitID, repo, timeout = 40, poll_freq_hz=2)
    # do ISR correction
    isr_corr_exposure = processExposure(exposure, repo=repo, bias=None, defects=None)
    
    if True: # display the image in firefly
        plt.close('all')
        disp = afwDisplay.Display(2, reopenPlot=True)
        disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
        disp.setImageColormap('gray')
        disp.scale('linear', 'zscale')
        disp.mtv(isr_corr_exposure, title='visit = {}'.format(visit_int))
        cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)
    
    # Find all sources in the image
    tab = afwTable.SourceTable.make(schema)
    result = sourceDetectionTask.run(tab, isr_corr_exposure, sigma=2.1)
    
    # Find the correct sources
    zeroth_order_star_BBox= lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(200,200)) 
    # wavelength solution is bad for the fiberSpectrograph, but close enough for this to work
    # can use the monochromator wavelength which is better, but that's not the correct way in the long run

    dispersion = (1/0.6358) # pixels/nm
    spectral_position_angle=0.0107 # radians clockwise from top

    # Find 0th and +/- 1 order peaks 
    sources = result.sources
    center_source, peak1, peak2 = findNarrowbandRonchiPeaks(sources, zeroth_order_star_BBox, wavelength, dispersion, spectral_position_angle)
    
    # Fit peaks
    # zeroth order
    # variables names are weird here because I can't think of a clever way to have -1 and +1 as variable names
    
    bbox0 = lsst.geom.Box2I.makeCenteredBox(center_source.getFootprint().getCentroid(), lsst.geom.Extent2I(100,100)) 
    peak0_subim = isr_corr_exposure.subset(bbox0)
    p0, x0 , y0 = fit2DGaussian(peak0_subim, plot=True)
    p0_x_CofM, p0_y_CofM = calc_CofM(peak0_subim) # 2167,3372

    # Calculate EE and CofM
    p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
    fit_data.append(p0_EE_rad80_pix)

#     # offset to test
#     bbox1 = lsst.geom.Box2I.makeCenteredBox(peak1.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak1_subim = isr_corr_exposure.subset(bbox1)
#     p1, x1 , y1 = fit2DGaussian(peak1_subim, plot=True)
#     p1_x_CofM, p1_y_CofM = calc_CofM(peak1_subim) # 2167,3372

#     # Calculate EE and CofM
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
    
#     fit_data.append(p1_EE_rad80_pix)
#     bbox2 = lsst.geom.Box2I.makeCenteredBox(peak2.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak2_subim = isr_corr_exposure.subset(bbox2)
#     p2, x2 , y2 = fit2DGaussian(peak2_subim, plot=False)
#     p2_x_CofM, p2_y_CofM = calc_CofM(peak2_subim) # 2167,3372

#     # Calculate EE and CofM
#     p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
#     p2_EE_rad50_pix, p2_EE_rad67_pix, p2_EE_rad80_pix = calc_encircled_energy(peak2_subim, plot=False) 

#     #  Now use multi_file_dataset
#     multi_file_dataset[i]['Gauss_x_peak'] = (p0.x_mean.value, p1.x_mean.value, p2.x_mean.value)
#     multi_file_dataset[i]['Gauss_y_peak'] = (p0.y_mean.value, p1.y_mean.value, p2.y_mean.value)
#     multi_file_dataset[i]['Gauss_xsigma_pix'] = (p0.x_stddev.value, p1.x_stddev.value, p2.x_stddev.value) 
#     multi_file_dataset[i]['Gauss_ysigma_pix'] = (p0.y_stddev.value, p1.y_stddev.value, p2.y_stddev.value)
#     multi_file_dataset[i]['x_CofM'] = (p0_x_CofM, p1_x_CofM, p2_x_CofM)
#     multi_file_dataset[i]['y_CofM'] = (p0_y_CofM, p1_y_CofM, p2_y_CofM)
#     multi_file_dataset[i]['EE50_pix'] = (p0_EE_rad50_pix, p1_EE_rad50_pix, p2_EE_rad50_pix)
#     multi_file_dataset[i]['EE67_pix'] = (p0_EE_rad67_pix, p1_EE_rad67_pix, p2_EE_rad67_pix)
#     multi_file_dataset[i]['EE80_pix'] = (p0_EE_rad80_pix, p1_EE_rad80_pix, p2_EE_rad80_pix)

NameError: name 'image_list' is not defined

2019-11-13 19:08:37,834 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 29 messages
2019-11-13 19:08:37,864 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, logMessage) falling behind; read 84 messages


In [163]:
#import lsst_python_utils

In [ ]:
fit_data

In [ ]:
# Fit a line to the profile
from scipy.optimize import curve_fit


xdata=focus_vals #[3:9]
ydata = fit_data #[3:9]

def parabola(x,b, x0, a):
    return b + a*(x-x0)**2 

popt,pcov = curve_fit(parabola, xdata, ydata, p0=[3.0, 69.5 , 1])

plt.close('all')
plt.figure(figsize=(13, 5))
plt.ylabel('Encircled Energy [pix]')
plt.plot(xdata, ydata, '.')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
plt.plot(x, parabola(x, *popt))
plt.title('Encircled Energy [pix]')
plt.xlabel('Focus position [mm]')
plt.show()
plt.close()

print('Best focus occurs at {} [mm]'.format(popt[1]))


In [ ]:
# Test loop of a bunch of biases

In [ ]:
n=1
start_time=np.empty(n)
camera_time=np.empty(n)
exposure_command_completed_time=np.empty(n)
archiver_time=np.empty(n)

for i in np.arange(n):
    # Take image
    print('starting image {}'.format(i))
    #First flush archiver event
    ATArchiver.evt_processingStatus.flush()

    group_id='Test'
    #ATCamera.cmd_takeImages.set(expTime=20.0, shutter=1, numImages=1, imageSequenceName=group_id)
    ATCamera.cmd_takeImages.set(expTime=0.0, shutter=0, numImages=1, imageSequenceName=group_id)

    start_time[i]=time.time()
    try:
        asyncio.get_event_loop().run_until_complete(ATCamera.cmd_takeImages.start(timeout=30))
    except salobj.AckError as ack_err:
        print(f"Failed with ack.result={ack_err.ack.result}")
    exposure_command_completed_time[i]=time.time()
    endReadout = asyncio.get_event_loop().run_until_complete(ATCamera.evt_endReadout.next(flush=True, timeout=30))
    print('ATCamera Wrote file {}'.format(endReadout.imageName) )
    camera_time[i]=time.time()

    test=asyncio.get_event_loop().run_until_complete(ATArchiver.evt_processingStatus.next(flush=False, timeout=60))
    print(test.description)
    archiver_time[i]=time.time()

In [ ]:
print('ATCamera time to write file [s]: {}'.format(camera_time[0]-exposure_command_completed_time[0]))
print('Archiver time to write file [s]: {}'.format(archiver_time[0]-exposure_command_completed_time[0]) )

In [ ]:
%matplotlib inline
x = camera_time-exposure_command_completed_time
plt.hist(x, bins=50)#, density=0.01)
plt.ylabel('Occurances')
plt.xlabel('Camera Timing [s]')
plt.title('Time of image writing from end of takeImage command completion')

In [ ]:
%matplotlib inline
y = archiver_time-exposure_command_completed_time
plt.hist(y, bins=50)#, density=0.01)
plt.ylabel('Occurances')
plt.xlabel('Archiver Timing [s]')
plt.title('Time of image writing from end of takeImage command completion')

In [ ]:
x0=x
y0=y

In [ ]:
camera_time[0]-exposure_command_completed_time[0]

In [ ]:
start_time = time.time()
await asyncio.sleep(1)
end_time = time.time()
print(start_time)
print(end_time)
print('net time [s]: {}'.format(end_time-start_time))

In [ ]:
np.arange(1)
